In [ ]:
from __future__ import print_function, division

%matplotlib inline
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['figure.figsize'] = (10,6)
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(context="poster")
import ipywidgets
import yt

import glob
import os
import warnings
import h5py

import numpy as np
import pandas as pd

from units import M_solar, m_proton, pc, yr, Myr, km, s, gamma

from injection_helpers import get_SNe

from visualize_helpers import \
    get_snapshot_filenames, \
    snapshot_filename_to_number, \
    total_mass_of_snapshot, \
    total_radial_momentum_of_snapshot, \
    map_to_all_snapshots, \
    get_snapshot_times, \
    total_kinetic_energy_of_snapshot, \
    total_internal_energy_of_snapshot, \
    load_snapshots, \
    plot_projected_density, \
    plot_sliced_field, \
    plot_phase_diagram, \
    field_type, \
    plot_profile, \
    yt_plot_saver, \
    mpl_plot_saver, \
    load_ds_from_ts

import MHD
    
from sql_helpers import add_simulation, \
    open_as_DataFrame

%load_ext autoreload
%autoreload 2

In [ ]:
# run_name = "cluster_cooling_100"
# run_name = "cluster_cooling_100-perturbed"
# run_name = "cluster_cooling_100-small_steps"
# run_name = "cluster_cooling_mhd_100"
run_name = "cluster_cooling_133"
# run_name = "cluster_cooling_200"
# run_name = "cluster_cooling_200-perturbed"
# run_name = "cluster_cooling_200-quantify_distributions"
# run_name = "cluster_cooling_200-small_steps"
# run_name = "cluster_cooling_200-small_steps-early"
# run_name = "cluster_cooling_mhd_200"
# run_name = "cluster_cooling_mhd_small_200"
# run_name = "cluster_cooling_mhd_large_200"
# run_name = "cluster_cooling_mhd_large_200-small_steps"
# run_name = "cluster_cooling_mhd_large_200-small_steps-early"
# run_name = "cluster_cooling_300"
# run_name = "cluster_cooling_400"
# run_name = "cluster_cooling_400-perturbed"
# run_name = "cluster_cooling_400-small_steps"
# run_name = "cluster_cooling_600"
# run_name = "single"
# run_name = "single_cooling"
# run_name = "double"
# run_name = "double_cooling"

run_dir = os.path.join(os.path.pardir, "runs", run_name)

inputs_dir  = os.path.join(run_dir, "inputs")
outputs_dir = os.path.join(run_dir, "outputs")

plots_dir = os.path.join(outputs_dir, "plots")
if not os.path.exists(plots_dir):
    print("making plots_dir")
    os.mkdir(plots_dir)

In [ ]:
save_plots = True

# Update / Load Database

In [ ]:
add_simulation(run_name, verbose=True)

df = open_as_DataFrame(run_name)
df.tail()

# Overview

In [ ]:
SNe = get_SNe(inputs_dir)
SN_times           = np.array([SN.time          for SN in SNe])
SN_ejecta_masses   = np.array([SN.ejecta_mass   for SN in SNe])
SN_ejecta_masses_Z = np.array([SN.ejecta_mass_Z for SN in SNe])

In [ ]:
ts = load_snapshots(outputs_dir)
snapshot_filenames = get_snapshot_filenames(outputs_dir)

snapshot_number_to_index_map = {snapshot_filename_to_number(filename) : i
                                for i,filename in enumerate(snapshot_filenames) }

uncompressed_snapshot_numbers = sorted(list(snapshot_number_to_index_map.keys()))

times_snapshots = df.time

print("Loaded {} uncompressed snapshots".format(len(ts)))

ds = load_ds_from_ts(ts, 0)
rho_0 = None


In [ ]:
times_snapshots

In [ ]:
SN_ejecta_masses

In [ ]:
print( "Length unit: ",   ds.length_unit)
print( "Time unit: ",     ds.time_unit)
print( "Mass unit: ",     ds.mass_unit)
print( "Velocity unit: ", ds.velocity_unit)

# What fields do I have access to?

In [ ]:
dd = ds.all_data()

In [ ]:
ds.field_list

In [ ]:
ds.derived_field_list

# Plot Summary Quantities

In [ ]:
energies = np.loadtxt(os.path.join(outputs_dir, "energy.txt"), ndmin=2)

times_statistics    = energies[:,0] * ds.time_unit
thermal_energies    = energies[:,1] * ds.mass_unit * (ds.velocity_unit)**2
potential_energies  = energies[:,2] * ds.mass_unit * (ds.velocity_unit)**2
kinetic_energies    = energies[:,3] * ds.mass_unit * (ds.velocity_unit)**2

times_statistics    = times_statistics.convert_to_cgs().value / Myr
thermal_energies    = thermal_energies.convert_to_cgs().value
potential_energies  = potential_energies.convert_to_cgs().value
kinetic_energies    = kinetic_energies.convert_to_cgs().value


total_mass    = energies[:,-6]


total_energies = thermal_energies + kinetic_energies

sorted_times_statistics_indices = np.argsort(times_statistics)

In [ ]:
_e_kin = df.e_kin
_e_int = df.e_int
_e_tot = _e_kin + _e_int

In [ ]:
thermal_energies[0:10]

In [ ]:
sns.rugplot(SN_times, color="k", linewidth=3)
plt.plot(times_statistics[sorted_times_statistics_indices],
         kinetic_energies[sorted_times_statistics_indices],
         label="statistics")
plt.plot(times_snapshots, _e_kin, label="snapshots", linestyle="dashed")
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$E_\mathrm{kin}$ $[\mathrm{ergs}]$")
plt.legend(loc="best")

plt.xlim(0,40)

plot_name = "E_kin"
if save_plots:
    mpl_plot_saver(plt.gcf(), plot_name, plots_dir)

In [ ]:
sns.rugplot(SN_times, color="k", linewidth=3)
plt.plot(times_statistics[sorted_times_statistics_indices],
         thermal_energies[sorted_times_statistics_indices],
         label="statistics",
        )
plt.plot(times_snapshots, _e_int, label="snapshots", linestyle="dashed")
plt.legend(loc="best")
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$E_\mathrm{int}$ $[\mathrm{ergs}]$")

plt.xlim(0,40)

plot_name = "E_int"
if save_plots:
    mpl_plot_saver(plt.gcf(), plot_name, plots_dir)

In [ ]:
sns.rugplot(SN_times, color="k", linewidth=3)
plt.plot(times_statistics[sorted_times_statistics_indices],
         total_energies[sorted_times_statistics_indices],
         label="statistics")
plt.plot(times_snapshots, _e_tot, label="snapshots", linestyle="dashed")
plt.legend(loc="best")
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$E_\mathrm{total}$ $[\mathrm{ergs}]$")

plt.xlim(0,40)

plot_name = "E_tot"
if save_plots:
    mpl_plot_saver(plt.gcf(), plot_name, plots_dir)

In [ ]:
sns.rugplot(SN_times, color="k", linewidth=3)
plt.plot(times_statistics[sorted_times_statistics_indices], 
         total_energies[sorted_times_statistics_indices] - total_energies[sorted_times_statistics_indices][0],
         label="statistics")
plt.plot(times_snapshots, _e_tot - _e_tot[0], label="snapshots", linestyle="dashed")
plt.legend(loc="best")
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$\Delta E_\mathrm{total}$ $[\mathrm{ergs}]$")

plt.xlim(0,40)

plot_name = "E_tot_delta"
if save_plots:
    mpl_plot_saver(plt.gcf(), plot_name, plots_dir)

# Plot Timing
Can be useful for noticing if the energy file doubles back on itself

In [ ]:
plt.plot(times_snapshots)
plt.xlabel("Snapshot #")
plt.ylabel("Time [Myr]")

plot_name = "times_snapshots"
if save_plots:
    mpl_plot_saver(plt.gcf(), plot_name, plots_dir)

In [ ]:
plt.plot(times_statistics)
plt.xlabel("Statistic Printout #")
plt.ylabel("Time [Myr]")

plot_name = "times_statistics"
if save_plots:
    mpl_plot_saver(plt.gcf(), plot_name, plots_dir)

## Mass Plots

WARNING: in "energy.txt" GIZMO only uses %g formatting; the change in total mass due to ejecta might be truncated 

In [ ]:
masses = df.mass

In [ ]:
sns.rugplot(SN_times, color="k", linewidth=3)
plt.plot(times_snapshots, masses - masses[0], 
         label="snapshots", linestyle="solid", drawstyle="steps-post")
plt.plot(SN_times, SN_ejecta_masses.cumsum(), 
         label="intended", linestyle="dashed", drawstyle="steps-post")
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$\Delta M$ $[M_\odot]$")
plt.legend(loc="best")

plt.xlim(0,40)

plot_name = "mass"
if save_plots:
    mpl_plot_saver(plt.gcf(), plot_name, plots_dir)

## Momentum Plots

In [ ]:
radial_momentum = df.momentum

In [ ]:
# radial_momentum / (100 * M_solar * km / s * SN_times.size)

In [ ]:
sns.rugplot(SN_times, color="k", linewidth=3)
plt.plot(times_snapshots, radial_momentum / (100 * M_solar * km / s))

plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$p$ $[100$ $M_\odot$ $\mathrm{km}$ $\mathrm{s}^{-1}]$")
plt.ylim(bottom=0)

plt.xlim(0,40)

plot_name = "momentum_total"
if save_plots:
    mpl_plot_saver(plt.gcf(), plot_name, plots_dir)

In [ ]:
sns.rugplot(SN_times, color="k", linewidth=3)
plt.plot(times_snapshots, radial_momentum / (100 * M_solar * km / s * SN_times.size))
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$p$ $[100$ $M_\odot$ $N_\mathrm{SNe}$ $\mathrm{km}$ $\mathrm{s}^{-1}]$")
plt.ylim(bottom=0)

plt.xlim(0,40)

plot_name = "momentum_normalized"
if save_plots:
    mpl_plot_saver(plt.gcf(), plot_name, plots_dir)

# get velocity distribution

In [ ]:
# ds = load_ds_from_ts(ts, snapshot_number_to_index_map[95])
# dd = ds.all_data()

In [ ]:
# vels = dd["all", "particle_velocity_magnitude"].value
# masses = dd["all", "particle_mass"].value / M_solar

In [ ]:
# plt.hist(vels, bins=np.logspace(-2, 6.5), weights=masses)

# plt.xscale("log")

# plt.xlabel("velocity magnitude")
# plt.ylabel("Mass within bin")

In [ ]:
# plt.hist(vels, bins=np.logspace(-2, 6.5), weights=masses, cumulative=-1)

# plt.xscale("log")

# plt.xlabel("velocity magnitude")
# plt.ylabel("Cumulative mass *above* a given velocity")

In [ ]:
# plt.hist(vels, bins=np.logspace(-2, 6.5), weights=masses, cumulative=-1, normed=True)

# plt.xscale("log")

# plt.xlabel("velocity magnitude")
# plt.ylabel("Percent mass *above* a given velocity")

In [ ]:
# with mpl.rc_context(rc={"figure.figsize": (10,10)}):
#     f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
#     ax1.hist(vels, bins=np.logspace(-2, 6.5), weights=masses)
#     ax1.set_ylabel(r"Mass per bin ($M_\odot$)")
#     ax1.axvline(100, linestyle="dashed", color="black")

#     ax2.hist(vels, bins=np.logspace(-2, 6.5), weights=masses, cumulative=-1, normed=True)
#     ax2.set_ylabel("Percent mass above\na given velocity")
#     ax2.axvline(100, linestyle="dashed", color="black")



#     plt.xscale("log")

#     plt.xlabel(r"velocity magnitude [cm s$^{-1}$]")
    
    


# Plot Snapshot Views

## but first, will I need B-field lines?

In [ ]:
if "mhd" in run_name.lower():
    ds = load_ds_from_ts(ts, snapshot_number_to_index_map[36])
    print("with MHD")
    field_line_filename = MHD.get_field_lines_filename_from_ds(ds, plots_dir)
    if not os.path.exists(field_line_filename):
        print("calculating magnetic field lines")
        df_lines = MHD.calculate_magnetic_field_lines(ds, plots_dir)
    else:
        print("field lines already calculated -- not recalculating")

## Projection Plot

In [ ]:
projection_widget = ipywidgets.interact_manual(plot_projected_density,
    ts = ipywidgets.fixed(ts),
    snapshot_number = ipywidgets.SelectionSlider(
        options=uncompressed_snapshot_numbers,
        value=uncompressed_snapshot_numbers[-1],
        ),
    snapshot_number_to_index_map = ipywidgets.fixed(snapshot_number_to_index_map),
    SN_times  = ipywidgets.fixed(SN_times),
    plots_dir = ipywidgets.fixed(plots_dir),
    save_plot = ipywidgets.fixed(save_plots),
    show_plot = ipywidgets.fixed(True),
    seaborn_style = ipywidgets.fixed("ticks"),
)
    
    

# Slice Plot

In [ ]:
slice_widget = ipywidgets.interact_manual(plot_sliced_field,
    ts = ipywidgets.fixed(ts),
    snapshot_number = ipywidgets.SelectionSlider(
        options=uncompressed_snapshot_numbers,
        value=uncompressed_snapshot_numbers[-1],
        ),
    snapshot_number_to_index_map = ipywidgets.fixed(snapshot_number_to_index_map),
    field = ipywidgets.Dropdown(options=list(field_type.keys()), value="density"),
    SN_times  = ipywidgets.fixed(SN_times),
    plots_dir = ipywidgets.fixed(plots_dir),
    save_plot = ipywidgets.fixed(save_plots),
    show_plot = ipywidgets.fixed(True),
    seaborn_style = ipywidgets.fixed("ticks"),
#     add_magnetic_field_lines = ipywidgets.fixed("mhd" in run_name.lower()),
    add_magnetic_field_lines = ipywidgets.fixed(False),
    font_size = ipywidgets.fixed(None),
    SlicePlot_kwargs = ipywidgets.fixed({"width":300}),
)


# Phase Plot

In [ ]:
phase_widget = ipywidgets.interact_manual(plot_phase_diagram,
    ts = ipywidgets.fixed(ts),
    snapshot_number = ipywidgets.SelectionSlider(
        options=uncompressed_snapshot_numbers,
        value=uncompressed_snapshot_numbers[-1],
        ),
    snapshot_number_to_index_map=ipywidgets.fixed(snapshot_number_to_index_map),
    SN_times=ipywidgets.fixed(SN_times),
    plots_dir=ipywidgets.fixed(plots_dir),
    weight_field=ipywidgets.Dropdown(options=["particle_mass", "cooling_rate"]), 
    save_plot=ipywidgets.fixed(save_plots),
    show_plot=ipywidgets.fixed(True),
    seaborn_style = ipywidgets.fixed("ticks"),
    field_type = ipywidgets.Dropdown(options=["all", "affected"]), 
    bins = ipywidgets.IntText(value=50)
)

In [ ]:
# with sns.axes_style("ticks"):
#     phase_widget.widget.result.set_zlim(("all", "particle_mass"),
#                                         1e-1,
#                                         6e6)
#     phase_widget.widget.result.show()

In [ ]:
# with sns.axes_style("ticks"):
#     phase_widget.widget.result.set_zlim(("all", "cooling_rate"),
#                                         3e43,
#                                         4e49)
#     phase_widget.widget.result.show()

# Radial Profile Plot

In [ ]:
if rho_0 is None:
    rho_0 = load_ds_from_ts(ts, 0).all_data()["all","density"].mean()

profile_widget = ipywidgets.interact_manual(plot_profile,
    ts = ipywidgets.fixed(ts),
    snapshot_number = ipywidgets.SelectionSlider(
        options=uncompressed_snapshot_numbers,
        value=uncompressed_snapshot_numbers[-1],
        ),
    snapshot_number_to_index_map = ipywidgets.fixed(snapshot_number_to_index_map),
    field = ipywidgets.Dropdown(options=list(field_type.keys()), value="density"),
    rho_0     = ipywidgets.fixed(0),
    plots_dir = ipywidgets.fixed(plots_dir),
    save_plot = ipywidgets.fixed(save_plots),
    show_plot = ipywidgets.fixed(True),
)